USECASE-1- AUTOMATED SUPPLIER SELECTION- AGENT-1

Pre-requisites:

#This notebook requires permissions to:
- create and delete Amazon IAM roles
- access Amazon Bedrock

#The following access are required for Sagemaker to carry on the usecase:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonBedrockFullAccess
- AmazonDynamoDBFullAccess

EDA and extraction of required columns

In [43]:
import pandas as pd

csv_file = r"Supply chain dataset - Supply chain dataset (2).csv"
data = pd.read_csv(csv_file)

def product_summary():

    headphones_data = data[data['ItemName'] == 'Camera']    #this usecase for headphones
  
    # extracting required columns
    relevant_columns = [
        'SupplierID', 'SupplierName', 'ItemID', 'ItemName', 'Price(rupees)',
        'DeliveryTime(days)', 'ReliabilityScore of delivery (percentage)', 
        'OrderVolume', 'SupplierCost(rupees)', 'ProfitMargin(rupees)', 
        'SalesData', 'MarketTrend', 'SeasonalDemand', 'ReorderQuantity', 'ReorderSchedule'
    ]
    
    headphones_info = headphones_data[relevant_columns]
    
    summaries = []
    for _, row in headphones_info.iterrows():
        summary = (
            f"Supplier {row['SupplierName']} (ID: {row['SupplierID']}) supplies item {row['ItemName']} "
            f"(ID: {row['ItemID']}) at a price of {row['Price(rupees)']} rupees. Delivery time is "
            f"{row['DeliveryTime(days)']} days with a reliability score of "
            f"{row['ReliabilityScore of delivery (percentage)']}%. Order volume is {row['OrderVolume']} units. "
            f"Supplier cost is {row['SupplierCost(rupees)']} rupees with a profit margin of "
            f"{row['ProfitMargin(rupees)']} rupees. Sales data shows {row['SalesData']} units sold, "
            f"market trend is {row['MarketTrend']}, seasonal demand is {row['SeasonalDemand']}. "
            f"Reorder quantity is {row['ReorderQuantity']} and reorder schedule is {row['ReorderSchedule']}."
        )
        summaries.append(summary)
    
    result = " ".join(summaries)
    return result


summary = product_summary()
print(summary)


Supplier Supplier_E (ID: 1005) supplies item Camera (ID: 6) at a price of 40000 rupees. Delivery time is 4 days with a reliability score of 81%. Order volume is 63 units. Supplier cost is 31525.32647 rupees with a profit margin of 8474.673534 rupees. Sales data shows 379 units sold, market trend is Stable, seasonal demand is Q3 (Jul-Sep). Reorder quantity is 93 and reorder schedule is 2024-08-23 7:04:28. Supplier Supplier_E (ID: 1005) supplies item Camera (ID: 6) at a price of 40000 rupees. Delivery time is 12 days with a reliability score of 82%. Order volume is 26 units. Supplier cost is 24946.35458 rupees with a profit margin of 15053.64542 rupees. Sales data shows 433 units sold, market trend is Stable, seasonal demand is Q3 (Jul-Sep). Reorder quantity is 54 and reorder schedule is 2024-08-31 7:04:28. Supplier Supplier_F (ID: 1006) supplies item Camera (ID: 6) at a price of 40000 rupees. Delivery time is 10 days with a reliability score of 89%. Order volume is 17 units. Supplier co

CREATING AGENT

In [3]:
!pip install boto3
!pip install opensearch-py
!pip install botocore
!pip install awscli
!pip install retrying

In [38]:
import boto3
import json
import time
import zipfile
import logging
import pprint
from io import BytesIO

iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
dynamodb_client = boto3.client('dynamodb')
dynamodb_resource = boto3.resource('dynamodb')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)





def invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=False, session_state=None):
    end_session: bool = False
    if not session_state:
        session_state = {}

    # invoke the agent API
    agent_response = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id,
        sessionId=session_id,
        enableTrace=enable_trace,
        endSession=end_session,
        sessionState=session_state
    )

    if enable_trace:
        logger.info(pprint.pprint(agent_response))

    event_stream = agent_response['completion']
    try:
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)


def create_agent_role(agent_name, agent_foundation_model, kb_id=None):
    agent_bedrock_allow_policy_name = f"{agent_name}-ba"
    agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
    # Create IAM policies for agent
    statements = [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
    # add Knowledge Base retrieve and retrieve and generate permissions if agent has KB attached to it
    if kb_id:
        statements.append(
            {
                "Sid": "QueryKB",
                "Effect": "Allow",
                "Action": [
                    "bedrock:Retrieve",
                    "bedrock:RetrieveAndGenerate"
                ],
                "Resource": [
                    f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{kb_id}"
                ]
            }
        )

    bedrock_agent_bedrock_allow_policy_statement = {
        "Version": "2012-10-17",
        "Statement": statements
    }

    bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)
    try:
        agent_bedrock_policy = iam_client.create_policy(
            PolicyName=agent_bedrock_allow_policy_name,
            PolicyDocument=bedrock_policy_json
        )
    except iam_client.exceptions.EntityAlreadyExistsException:
        agent_bedrock_policy = iam_client.get_policy(
            PolicyArn=f"arn:aws:iam::{account_id}:policy/{agent_bedrock_allow_policy_name}"
        )
                    
    # Create IAM Role for the agent and attach IAM policies
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)
    try:
        agent_role = iam_client.create_role(
            RoleName=agent_role_name,
            AssumeRolePolicyDocument=assume_role_policy_document_json
        )

        # Pause to make sure role is created
        time.sleep(10)
    except iam_client.exceptions.EntityAlreadyExistsException:
        agent_role = iam_client.get_role(
            RoleName=agent_role_name,
        )

    iam_client.attach_role_policy(
        RoleName=agent_role_name,
        PolicyArn=agent_bedrock_policy['Policy']['Arn']
    )
    return agent_role


def delete_agent_roles_and_policies(agent_name, kb_policy_name):
    agent_bedrock_allow_policy_name = f"{agent_name}-ba"
    agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
    dynamodb_access_policy_name = f'{agent_name}-dynamodb-policy'
    lambda_function_role = f'{agent_name}-lambda-role'

    for policy in [agent_bedrock_allow_policy_name, kb_policy_name]:
        try:
            iam_client.detach_role_policy(
                RoleName=agent_role_name,
                PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}'
            )
        except Exception as e:
            print(f"Could not detach {policy} from {agent_role_name}")
            print(e)

    for policy in [dynamodb_access_policy_name]:
        try:
            iam_client.detach_role_policy(
                RoleName=lambda_function_role,
                PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}'
            )
        except Exception as e:
            print(f"Could not detach {policy} from {lambda_function_role}")
            print(e)

    try:
        iam_client.detach_role_policy(
            RoleName=lambda_function_role,
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )
    except Exception as e:
        print(f"Could not detach AWSLambdaBasicExecutionRole from {lambda_function_role}")
        print(e)

    for role_name in [agent_role_name, lambda_function_role]:
        try:
            iam_client.delete_role(
                RoleName=role_name
            )
        except Exception as e:
            print(f"Could not delete role {role_name}")
            print(e)

    for policy in [agent_bedrock_allow_policy_name, kb_policy_name, dynamodb_access_policy_name]:
        try:
            iam_client.delete_policy(
                PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}'
            )
        except Exception as e:
            print(f"Could not delete policy {policy}")
            print(e)


def clean_up_resources(
        table_name, lambda_function, lambda_function_name, agent_action_group_response, agent_functions,
        agent_id, kb_id, alias_id
):
    action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
    action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']
    # Delete Agent Action Group, Agent Alias, and Agent
    try:
        bedrock_agent_client.update_agent_action_group(
            agentId=agent_id,
            agentVersion='DRAFT',
            actionGroupId= action_group_id,
            actionGroupName=action_group_name,
            actionGroupExecutor={
                'lambda': lambda_function['FunctionArn']
            },
            functionSchema={
                'functions': agent_functions
            },
            actionGroupState='DISABLED',
        )
        bedrock_agent_client.disassociate_agent_knowledge_base(
            agentId=agent_id,
            agentVersion='DRAFT',
            knowledgeBaseId=kb_id
        )
        bedrock_agent_client.delete_agent_action_group(
            agentId=agent_id,
            agentVersion='DRAFT',
            actionGroupId=action_group_id
        )
        bedrock_agent_client.delete_agent_alias(
            agentAliasId=alias_id,
            agentId=agent_id
        )
        bedrock_agent_client.delete_agent(agentId=agent_id)
        print(f"Agent {agent_id}, Agent Alias {alias_id}, and Action Group have been deleted.")
    except Exception as e:
        print(f"Error deleting Agent resources: {e}")

    # Delete Lambda function
    try:
        lambda_client.delete_function(FunctionName=lambda_function_name)
        print(f"Lambda function {lambda_function_name} has been deleted.")
    except Exception as e:
        print(f"Error deleting Lambda function {lambda_function_name}: {e}")

    # Delete DynamoDB table
    try:
        dynamodb_client.delete_table(TableName=table_name)
        print(f"Table {table_name} is being deleted...")
        waiter = dynamodb_client.get_waiter('table_not_exists')
        waiter.wait(TableName=table_name)
        print(f"Table {table_name} has been deleted.")
    except Exception as e:
        print(f"Error deleting table {table_name}: {e}")

In [39]:
import time
import boto3
import logging
import ipywidgets as widgets
import uuid




In [40]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-east-1', '199328835594')

In [14]:
suffix = f"{region}-{account_id}"
agent_name = 'supplier-reco-awshack'
agent_bedrock_allow_policy_name = f"{agent_name}-ba"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

agent_description = "Agent in charge of going through complete prodcut and supplier data and then refer best supplier for a certain product. also give the reason for selecting the particular supplier"
agent_instruction = """
You are a supply chain management agent. you go through the  complete product and supplier data and then refer best supplier for a certain product. also give the reason for selecting the particular supplier
"""

agent_action_group_description = """
Creating supplier recommendations"""    

agent_action_group_name = "Supplierrecommendation"

In [15]:
agent_foundation_model = 'anthropic.claude-3-sonnet-20240229-v1:0'
agent_foundation_model

'anthropic.claude-3-sonnet-20240229-v1:0'

In [16]:
agent_role = create_agent_role(agent_name, agent_foundation_model)

In [17]:
agent_role

{'Role': {'Path': '/',
  'RoleName': 'AmazonBedrockExecutionRoleForAgents_supplier-reco-awshack',
  'RoleId': 'AROAS42HMVAFA3DVR74SL',
  'Arn': 'arn:aws:iam::199328835594:role/AmazonBedrockExecutionRoleForAgents_supplier-reco-awshack',
  'CreateDate': datetime.datetime(2024, 8, 26, 4, 58, 33, tzinfo=tzlocal()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Effect': 'Allow',
     'Principal': {'Service': 'bedrock.amazonaws.com'},
     'Action': 'sts:AssumeRole'}]}},
 'ResponseMetadata': {'RequestId': 'cf43dd26-8271-4aab-b5b5-c4b657b7b479',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 26 Aug 2024 04:58:32 GMT',
   'x-amzn-requestid': 'cf43dd26-8271-4aab-b5b5-c4b657b7b479',
   'content-type': 'text/xml',
   'content-length': '869'},
  'RetryAttempts': 0}}

In [18]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

{'ResponseMetadata': {'RequestId': '8ee2aa8d-ad38-4010-910f-391dea72cb26',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 26 Aug 2024 05:00:16 GMT',
   'content-type': 'application/json',
   'content-length': '881',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8ee2aa8d-ad38-4010-910f-391dea72cb26',
   'x-amz-apigw-id': 'dGbLKG1UoAMEhIA=',
   'x-amzn-trace-id': 'Root=1-66cc0be0-770750d23482dc7b634c85f7'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-east-1:199328835594:agent/N7NLIWSKIK',
  'agentId': 'N7NLIWSKIK',
  'agentName': 'supplier-reco-awshack',
  'agentResourceRoleArn': 'arn:aws:iam::199328835594:role/AmazonBedrockExecutionRoleForAgents_supplier-reco-awshack',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 8, 26, 5, 0, 16, 860499, tzinfo=tzlocal()),
  'description': 'Agent in charge of going through complete prodcut and supplier data and then refer best supplier for a certain product. also give the reason for selecti

In [19]:
agent_id = response['agent']['agentId']
print("The agent id is:",agent_id)

The agent id is: N7NLIWSKIK


In [27]:
#create and deploy lambda here

In [20]:
agent_functions = [
    
    {
        'name': 'create_recommendation',
        'description': 'creating best supplier for each product',
        'parameters': {
            "SupplierName": {
                "description": "Name of the supplier",
                "required": True,
                "type": "string"
            },
            "Product_name": {
                "description": "Name of the prodcut supplied",
                "required": True,
                "type": "string"
            },
            "Reason_for_choice": {
                "description": "Reason why this supplier is choosen as the best one for this product",
                "required": True,
                "type": "string"
            }
        }
    },
    
]

In [21]:
# Pause to make sure agent is created
time.sleep(30)

# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda':'arn:aws:lambda:us-east-1:199328835594:function:best-supplier-lambda'
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [22]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': '4b704651-92fc-4aaa-baf1-f3c22aceba4f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 26 Aug 2024 05:28:03 GMT',
   'content-type': 'application/json',
   'content-length': '903',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4b704651-92fc-4aaa-baf1-f3c22aceba4f',
   'x-amz-apigw-id': 'dGfPnFR2IAMEnkA=',
   'x-amzn-trace-id': 'Root=1-66cc1263-55cf0c924c72c20748f06726'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:199328835594:function:best-supplier-lambda'},
  'actionGroupId': 'DNTJPEPRO9',
  'actionGroupName': 'Supplierrecommendation',
  'actionGroupState': 'ENABLED',
  'agentId': 'N7NLIWSKIK',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2024, 8, 26, 5, 28, 3, 828882, tzinfo=tzlocal()),
  'description': '\nCreating supplier recommendations',
  'functionSchema': {'functions': [{'description': 'creating best supplier for each product',
     'name': 'create_

In [23]:
# Create allow to invoke permission on lambda
lambda_client = boto3.client('lambda')
try:
    response = lambda_client.add_permission(
        FunctionName= 'best-supplier-lambda',
        StatementId=f'allow_bedrock_{agent_id}',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except Exception as e:
    print(e)

{'ResponseMetadata': {'RequestId': '4416afd7-1bdb-4323-95ed-90c6b0bb4eec', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Mon, 26 Aug 2024 05:28:12 GMT', 'content-type': 'application/json', 'content-length': '359', 'connection': 'keep-alive', 'x-amzn-requestid': '4416afd7-1bdb-4323-95ed-90c6b0bb4eec'}, 'RetryAttempts': 0}, 'Statement': '{"Sid":"allow_bedrock_N7NLIWSKIK","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-east-1:199328835594:function:best-supplier-lambda","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-east-1:199328835594:agent/N7NLIWSKIK"}}}'}


In [24]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(30)

{'ResponseMetadata': {'RequestId': '724d3caa-c75c-4862-83c4-f3825e3f5ec3', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Mon, 26 Aug 2024 05:28:25 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '724d3caa-c75c-4862-83c4-f3825e3f5ec3', 'x-amz-apigw-id': 'dGfS8HZnIAMEpaA=', 'x-amzn-trace-id': 'Root=1-66cc1278-7fedeaf41f3f9e4f52d8ed44'}, 'RetryAttempts': 0}, 'agentId': 'N7NLIWSKIK', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 8, 26, 5, 28, 25, 8739, tzinfo=tzlocal())}


In [44]:
alias_id = 'TSTALIASID'
agent_id = 'N7NLIWSKIK'

#INVOKING AGENT AND RESULT

In [45]:
%%time
session_id:str = str(uuid.uuid1())
query = summary + "Select the best supplier for camera based on the content. also give the reason for the recommendation"

response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Based on the analysis of the given data, I recommend Supplier_F (ID: 1006) as the best supplier for the Camera product. The key reasons are:

- Supplier_F has the highest reliability score of 89%, indicating very dependable and consistent delivery performance.
- They have a relatively short delivery time of 10-14 days, allowing reasonably quick fulfillment of orders.
- Supplier_F can handle decent order volumes like 17-72 units per order.
- Their supplier cost is competitive at around 26,000-29,000 rupees with a good profit margin over 10,000 rupees, contributing to better profitability.
- The sales data shows high sales volumes of 838-991 units with an increasing market trend, suggesting strong demand for their Camera products.
- Their reorder quantities of 40-65 units and reorder schedules seem manageable for maintaining inventory levels.

Considering the reliability, delivery times, costs, sales performance and ability to meet order/reorder requirements, Supplier_F emerges as the be